# Documentation

In [1]:
from IPython.core.display import HTML
HTML("""<style>
    .MathJax_Display { font-size: 23px; }
</style>""")

$$
\theta = \theta + \alpha (r + \gamma max_{a'} Q(s',a') - Q(s,a)) \nabla_{\theta} Q(s,a)
$$

# Setup

In [2]:
"""
Add parent directorys to current path
"""
import os.path
import sys
p = os.path.abspath('..')
if p not in sys.path:
    sys.path.insert(0,p)
p = os.path.abspath('../..')
if p not in sys.path:
    sys.path.insert(0,p)
    
    
"""
Add tiger-env directory to current path
Still not sure why this is needed.
"""
p = os.path.abspath('../../../custom_envs/gym-tiger')
if p not in sys.path:
    sys.path.insert(0, p)
    

"""
Enable hot-reloading
"""    
from notebook_utils import import_module_by_name, reload_module_by_name

def reload():
    reload_module_by_name('experiments..model', 'DQN')
    global DQN
    from experiments.dqn.model import DQN
    
    
import gym
import gym_tiger
import matplotlib.pyplot as plt
from experiments.dqn.model import DQN, play_one, main, running_avg, plot_running_avg
from tabulate import tabulate


OBS_GROWL_LEFT = [1, 0, 0, 0]
OBS_GROWL_RIGHT = [0, 1, 0, 0]
OBS_START = [0, 0, 1, 0]
OBS_END = [0, 0, 0, 1]

ACTION_OPEN_LEFT = 0
ACTION_OPEN_RIGHT = 1
ACTION_LISTEN = 2
ACTION_MAP = {
    ACTION_OPEN_LEFT: 'OPEN_LEFT',
    ACTION_OPEN_RIGHT: 'OPEN_RIGHT',
    ACTION_LISTEN: 'LISTEN',
}

# Play 1000 Episodes with 100% observation accuracy

In [3]:
"""
Remove old tensorboard log files
"""
!rm -rf ./logs

import tensorflow as tf

env = gym.make('Tiger-v0')
env.__init__(reward_tiger=-100, reward_gold=10, reward_listen=-1,
            obs_accuracy=1)
gamma = 0.99
hidden_layer_sizes = [25, 25]
batch_size = 2
copy_period = 25
N=1000

session = tf.InteractiveSession()
D = env.observation_space.n
K = env.action_space.n
model = DQN('main', session, D, K, hidden_layer_sizes, gamma,
            batch_size=batch_size,
            log_summaries=True, logs_dir='./logs')
tmodel = DQN('target', session, D, K, hidden_layer_sizes, gamma,
             batch_size=batch_size)
init = tf.global_variables_initializer()
session.run(init)

totalrewards = np.zeros(N)
for n in range(N):
    eps = 1.0/np.sqrt(n+1)
#     eps = 1/(n+1)**(1/5)
    totalreward = play_one(env, model, tmodel, eps, gamma,
                           copy_period, max_steps=10)
    totalrewards[n] = totalreward
    if n % 100 == 0:
        ravg = running_avg(totalrewards, n)
        print('episode:', n,
              'total reward:', totalreward,
              'eps:', eps,
              'avg reward (last 100):', ravg)


print('avg reward for last 100 episodes:', totalrewards[-100:].mean())
print('total steps:', totalrewards.sum())

# plt.plot(totalrewards)
# plt.title("Rewards")
# plt.show()

# plot_running_avg(totalrewards)



"""
Inspect final Q values
"""
actions = model.predict([OBS_START,
                         OBS_GROWL_LEFT,
                         OBS_GROWL_RIGHT])
obs = np.array(['START', 'GROWL_LEFT', 'GROWL_RIGHT']).reshape(-1,1)
actions = np.hstack([obs, actions])
print(tabulate(actions, headers=list(ACTION_MAP.values())))

Instructions for updating:
Colocations handled automatically by placer.


/Users/jblandin/miniconda3/envs/research/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


episode: 0 total reward: -274 eps: 1.0 avg reward (last 100): -274.0
episode: 100 total reward: 1 eps: 0.09950371902099892 avg reward (last 100): -156.92079207920793
episode: 200 total reward: -10 eps: 0.07053456158585983 avg reward (last 100): -30.475247524752476
episode: 300 total reward: -10 eps: 0.0576390417704235 avg reward (last 100): -23.722772277227723
episode: 400 total reward: -10 eps: 0.04993761694389223 avg reward (last 100): -25.574257425742573
episode: 500 total reward: -109 eps: 0.04467670516087703 avg reward (last 100): -26.11881188118812
episode: 600 total reward: -10 eps: 0.04079085082240021 avg reward (last 100): -23.396039603960396
episode: 700 total reward: -10 eps: 0.0377694787300249 avg reward (last 100): -19.03960396039604
episode: 800 total reward: -10 eps: 0.03533326266687867 avg reward (last 100): -22.96039603960396
episode: 900 total reward: -10 eps: 0.03331483023263848 avg reward (last 100): -21.217821782178216
avg reward for last 100 episodes: -15.83
total

# Play 1000 Episodes with 92% observation accuracy

In [58]:
"""
Remove old tensorboard log files
"""
!rm -rf ./logs

import tensorflow as tf

env = gym.make('Tiger-v0')
env.__init__(reward_tiger=-100, reward_gold=10, reward_listen=-1,
            obs_accuracy=.92)
gamma = 0.99
hidden_layer_sizes = [25, 25]
batch_size = 2
copy_period = 25
N=1000

session = tf.InteractiveSession()
D = env.observation_space.n
K = env.action_space.n
model = DQN('main', session, D, K, hidden_layer_sizes, gamma,
            batch_size=batch_size,
            log_summaries=True, logs_dir='./logs')
tmodel = DQN('target', session, D, K, hidden_layer_sizes, gamma,
             batch_size=batch_size)
init = tf.global_variables_initializer()
session.run(init)

totalrewards = np.zeros(N)
for n in range(N):
    eps = 1.0/np.sqrt(n+1)
#     eps = 1/(n+1)**(1/5)
    totalreward = play_one(env, model, tmodel, eps, gamma,
                           copy_period, max_steps=10)
    totalrewards[n] = totalreward
    if n % 100 == 0:
        ravg = running_avg(totalrewards, n)
        print('episode:', n,
              'total reward:', totalreward,
              'eps:', eps,
              'avg reward (last 100):', ravg)


print('avg reward for last 100 episodes:', totalrewards[-100:].mean())
print('total steps:', totalrewards.sum())

# plt.plot(totalrewards)
# plt.title("Rewards")
# plt.show()

# plot_running_avg(totalrewards)



"""
Inspect final Q values
"""
actions = model.predict([OBS_START,
                         OBS_GROWL_LEFT,
                         OBS_GROWL_RIGHT])
obs = np.array(['START', 'GROWL_LEFT', 'GROWL_RIGHT']).reshape(-1,1)
actions = np.hstack([obs, actions])
print(tabulate(actions, headers=list(ACTION_MAP.values())))

episode: 0 total reward: 10 eps: 1.0 avg reward (last 100): 10.0
episode: 100 total reward: 10 eps: 0.09950371902099892 avg reward (last 100): -46.722772277227726
episode: 200 total reward: 10 eps: 0.07053456158585983 avg reward (last 100): -40.98019801980198
episode: 300 total reward: -100 eps: 0.0576390417704235 avg reward (last 100): -24.465346534653467
episode: 400 total reward: -10 eps: 0.04993761694389223 avg reward (last 100): -21.801980198019802
episode: 500 total reward: -10 eps: 0.04467670516087703 avg reward (last 100): -18.85148514851485
episode: 600 total reward: -10 eps: 0.04079085082240021 avg reward (last 100): -17.99009900990099
episode: 700 total reward: -10 eps: 0.0377694787300249 avg reward (last 100): -17.91089108910891
episode: 800 total reward: -10 eps: 0.03533326266687867 avg reward (last 100): -19.059405940594058
episode: 900 total reward: -10 eps: 0.03331483023263848 avg reward (last 100): -15.564356435643564
avg reward for last 100 episodes: -14.1
total steps

# Play 1000 Episodes with 85% observation accuracy

In [59]:
"""
Remove old tensorboard log files
"""
!rm -rf ./logs

import tensorflow as tf

env = gym.make('Tiger-v0')
env.__init__(reward_tiger=-100, reward_gold=10, reward_listen=-1,
            obs_accuracy=.85)
gamma = 0.99
hidden_layer_sizes = [25, 25]
batch_size = 2
copy_period = 25
N=1000

session = tf.InteractiveSession()
D = env.observation_space.n
K = env.action_space.n
model = DQN('main', session, D, K, hidden_layer_sizes, gamma,
            batch_size=batch_size,
            log_summaries=True, logs_dir='./logs')
tmodel = DQN('target', session, D, K, hidden_layer_sizes, gamma,
             batch_size=batch_size)
init = tf.global_variables_initializer()
session.run(init)

totalrewards = np.zeros(N)
for n in range(N):
    eps = 1.0/np.sqrt(n+1)
#     eps = 1/(n+1)**(1/5)
    totalreward = play_one(env, model, tmodel, eps, gamma,
                           copy_period, max_steps=10)
    totalrewards[n] = totalreward
    if n % 100 == 0:
        ravg = running_avg(totalrewards, n)
        print('episode:', n,
              'total reward:', totalreward,
              'eps:', eps,
              'avg reward (last 100):', ravg)


print('avg reward for last 100 episodes:', totalrewards[-100:].mean())
print('total steps:', totalrewards.sum())

# plt.plot(totalrewards)
# plt.title("Rewards")
# plt.show()

# plot_running_avg(totalrewards)



"""
Inspect final Q values
"""
actions = model.predict([OBS_START,
                         OBS_GROWL_LEFT,
                         OBS_GROWL_RIGHT])
obs = np.array(['START', 'GROWL_LEFT', 'GROWL_RIGHT']).reshape(-1,1)
actions = np.hstack([obs, actions])
print(tabulate(actions, headers=list(ACTION_MAP.values())))

episode: 0 total reward: 8 eps: 1.0 avg reward (last 100): 8.0
episode: 100 total reward: -100 eps: 0.09950371902099892 avg reward (last 100): -46.71287128712871
episode: 200 total reward: -100 eps: 0.07053456158585983 avg reward (last 100): -46.633663366336634
episode: 300 total reward: -10 eps: 0.0576390417704235 avg reward (last 100): -30.0
episode: 400 total reward: -10 eps: 0.04993761694389223 avg reward (last 100): -18.415841584158414
episode: 500 total reward: -10 eps: 0.04467670516087703 avg reward (last 100): -19.653465346534652
episode: 600 total reward: -10 eps: 0.04079085082240021 avg reward (last 100): -19.81188118811881
episode: 700 total reward: 7 eps: 0.0377694787300249 avg reward (last 100): -14.049504950495049
episode: 800 total reward: -10 eps: 0.03533326266687867 avg reward (last 100): -11.405940594059405
episode: 900 total reward: -10 eps: 0.03331483023263848 avg reward (last 100): -21.316831683168317
avg reward for last 100 episodes: -19.22
total steps: -24697.0
 